# CORD-19 NLP Workflow

## Dataset Overview

The CORD-19 dataset [url](https://www.semanticscholar.org/cord19) is an open dataset curated by the Allen Institute for AI intended to facilitate NLP research efforts pertaining to COVID-19. It provides a compilation of relevant peer-reviewed research publications from queries to the following sources: 

- PubMed's open access corpus
- The WHO's open access corpus
- bioRxiv and medRxiv preprints

The CORD-19 dataset is updated daily, with a .csv of publication metadata provided as well as full text (as available) from articles in JSON format. This dataset is part of a number of NLP research efforts, and is a current Kaggle challenge dataset [url](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge).

## Workflow Overview

Our goal is to provide a workflow for automating data preparation, processing, and resources for analyses of this dataset. We chose this dataset in part due to its timeliness and relevance, as well as its recurring updates; indeed, it lends itself well to automation. Having a workflow of this sort also foments experiment repeatability and provenance, two crucial elements in accountable AI. 

With this in mind, we provide below our means for:

- Acquiring the dataset
- Preparing and processing the data of interest
- Generating summary information 

We chose to focus on the abstract details in the metadata.csv file provided by CORD-19 and the associated full text provided in the pdf_json directory. The former file provides full abstracts from publications, while the latter files provide complete text, both of which we utilize for unsupervised topic modeling via LDA.

First we'll import our libraries. We'll use spacy for extracting tokens from the abstracts, nltk for its stopwords corpus as we found it to be more comprehensive than spacy's, gensim to generate a corpus and LDA model, and plotting libraries for visualizing summaries.  


In [ ]:
import csv
import os
import json
from collections import defaultdict
import tqdm
from itertools import islice

import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
spacy.prefer_gpu()

from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_lg 

from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

import nltk
from nltk.corpus import stopwords 
import pyLDAvis.gensim

We'll first acquire the dataset and extract it. 

In [ ]:
dataset_date='2020-07-16'
dataset_filename = 'cord-19_' + dataset_date + '.tar.gz'
dataset_url = 'https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/historical_releases/' + dataset_filename
if os.path.exists(dataset_date) != True:
    !wget -nc dataset_url
    !tar -xzf dataset_filename
os.chdir(dataset_date)

Next we'll load spacy, and add our own set of words to the NLTK stopwords set, then update spacy's default stop_words set. By doing this, we reduce the potential for generally unique but dataset specific uninteresting and expected words to appear in our topic lists.

In [ ]:
nlp= spacy.load("en_core_web_lg")
stop_words = stopwords.words('english')
stop_words.extend(['sars','covid-19', 'cov-2','=','from', 'subject', 're', 'edu', 'use', 'not', 
                   'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 
                   'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 
                   'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 
                   'line', 'even', 'also', 'may', 'take', 'come'])

nlp.Defaults.stop_words.update(stop_words)

Much like we are looking to automate this work on a macro level, we're also fans of the pipeline infrastructure spacy provides, so we define two steps of our NLP pipeline below and add them to our spacy instance. Namely, preprocessing the text provided by splitting into tokens and removing punctuation, stop words, and numerics. 

In [ ]:
def lemmatize(doc):
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
    
def clean_tokens(doc):
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True and (len(token)<=4)!=True]
    return doc

In [ ]:
nlp.add_pipe(lemmatize,name='lemmatize',after='ner')
nlp.add_pipe(clean_tokens, name="cleanup", last=True)

Now we're ready to extract our abstract and publication data from metadata.csv, as well as the full text from the json-ified publication files. We then process each extracted piece of data through our spacy pipeline, then add it to a running list of abstracts and full text. For purposes of resource reduction and speed, the `dataset_limit` variable is provided below. This can be modified to process the entire dataset accordingly. 

In [ ]:
fulltext_list = []
abstract_list = []
nlp.max_length = 2000000
dataset_limit = 100

with open('metadata.csv') as f_in:
    reader = csv.DictReader(f_in)
    for row in tqdm.tqdm_notebook(islice(reader, 0, dataset_limit),total=dataset_limit):
        abstract = row['abstract']
        all_text = []
        if row['pdf_json_files']:
            for json_path in row['pdf_json_files'].split('; '):
                with open(json_path) as f_json:
                    full_text_dict = json.load(f_json)
                    for paragraph_dict in full_text_dict['body_text']:
                        paragraph_text = paragraph_dict['text']
                        section_name = paragraph_dict['section']
                        all_text.append(paragraph_text)
        nlp_abstract = nlp(abstract)
        abstract_list.append(nlp_abstract)
        nlp_fulltext = nlp(u' '.join(all_text))
        fulltext_list.append(nlp_fulltext)                                

With our document lists in place, we are ready to generated our word dictionary and corpus for abstracts and full text, which we then can use to (finally) create an LDA model of our documents!

In [ ]:
def generate_corpus(doc_list):
    words=corpora.Dictionary(doc_list)
    corpus = [words.doc2bow(doc) for doc in doc_list]
    return [words,corpus]

def generate_lda_model(words,corpus):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=words,
                                            num_topics=10, 
                                            random_state=2,
                                            update_every=1,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)  
    return lda_model

In [ ]:
gen_corp = generate_corpus(abstract_list)
abstract_words=gen_corp[0]
abstract_corpus=gen_corp[1]
abstract_lda_model = generate_lda_model(abstract_words,abstract_corpus)

In [ ]:
gen_corp = generate_corpus(fulltext_list)
fulltext_words=gen_corp[0]
fulltext_corpus=gen_corp[1]
fulltext_lda_model = generate_lda_model(fulltext_words,fulltext_corpus)

As we now have LDA models generated, we can visualize our analyses. Below we generate word clouds for our topics. Followed by a pyLDAvis gensim specific topic modeling plot showing word strength within topics and distance between topic clusters.

In [ ]:
def plotWordcloud(lda_model,stop_words):
    cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]

    cloud = WordCloud(stopwords=stop_words,
                      background_color='white',
                      width=2500,
                      height=1800,
                      max_words=10,
                      colormap='tab10',
                      color_func=lambda *args, **kwargs: cols[i],
                      prefer_horizontal=1.0)

    topics = lda_model.show_topics(formatted=False)

    fig, axes = plt.subplots(2, 2, figsize=(10,10), sharex=True, sharey=True)

    for i, ax in enumerate(axes.flatten()):
        fig.add_subplot(ax)
        topic_words = dict(topics[i][1])
        cloud.generate_from_frequencies(topic_words, max_font_size=300)
        plt.gca().imshow(cloud)
        plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
        plt.gca().axis('off')


    plt.subplots_adjust(wspace=0, hspace=0)
    plt.axis('off')
    plt.margins(x=0, y=0)
    plt.tight_layout()
    return plt

In [ ]:
abstract_wordcloud = plotWordcloud(abstract_lda_model, stop_words)

In [ ]:
fulltext_wordcloud = plotWordcloud(fulltext_lda_model, stop_words)

In [ ]:
pyLDAvis.enable_notebook()                           

In [ ]:
abstract_ldavis = pyLDAvis.gensim.prepare(abstract_lda_model, abstract_corpus, dictionary=abstract_lda_model.id2word)
abstract_ldavis

In [ ]:
fulltext_ldavis = pyLDAvis.gensim.prepare(fulltext_lda_model, fulltext_corpus, dictionary=fulltext_lda_model.id2word)
fulltext_ldavis

Next we'll move relevant pieces of this code to containerized applciations contained within OpenShift, which will then be orchestrated with an Argo workflow.